### 구현 방식
1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면(인풋이 길면) 답변 생성이 오래걸린다.
3. (쪼갠 문서를) 임베딩 -> 벡터 DB에 저장
4. 질문이 있을 때 벡터 DB에서 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [5]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,    # 문서를 쪼갤 때 하나의 chunk가 가지는 token 수
    chunk_overlap=200   # 쪼갠 내용에서 겹치는 정도
)

loader = Docx2txtLoader("./tax.docx")
document_list = loader.load_and_split(text_splitter=text_splitter)

In [6]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

In [8]:
""" 이 부분만 변경하면 된다. """

# from langchain_chroma import Chroma

# database = Chroma(
#     collection_name="chroma-tax",
#     persist_directory="./chroma",
#     embedding_function=embedding
# )

import os, time
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
pinecone_api_key = os.environ.get("PINECONE_API_KEY")

index_name = "tax-index"
pc = Pinecone(api_key=pinecone_api_key)

database = PineconeVectorStore.from_documents(document_list, embedding, index_name)

TypeError: VectorStore.from_documents() takes 3 positional arguments but 4 were given

In [ ]:
# %pip install -qU langchain-pinecone pinecone-notebooks
# %pip list | findstr "^pinecone-notebooks"

In [4]:
# database에 query를 날려보자
query = "연봉 5천만 원인 직장인의 소득세는 얼마인가요?"

# similarity_search: database 생성 시 넣은 embedding을 활용해 유사도 검색
# 유사도 검색을 통해 답변을 가져온다.
# retrieved_docs = database.similarity_search(query, k=3) # k: return 하는 답변의 수

In [5]:
# 문서를 가져왔으니 이를 토대로 LLM에 질의를 한다.
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")

# [Identity] --> 페르소나 설정(이게 있으면 답변이 더 좋다고 한다.)
# prompt = f"""
#     [Identity]
#     - 당신은 최고의 한국 소득세 전문가 입니다.
#     - [Context]를 참고해서 사용자의 질문에 답변해주세요.

#     [Context]
#     {retrieved_docs}

#     Question: {query}
# """
# ai_message = llm.invoke(prompt)
# ai_message

In [6]:
from langchain import hub
# langchain langchainhub --> prompt 제공
prompt = hub.pull("rlm/rag-prompt") # langchain에 default로 있는 prompt 사용

In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt" : prompt}
)
ai_message = qa_chain({"query" : query})
ai_message